# Recurrent recommender networks
## Trabalho final da disciplina de Introdução à Machine Learning - UFRJ 2022.2

Baseado no artigo [Wu, Chao-Yuan, et al. 'Recurrent recommender networks.' 2017.](https://dl.acm.org/doi/epdf/10.1145/3018661.3018689)

## Bibliotecas

In [1]:
import pandas as pd
import numpy as np

import tensorflow.compat.v1 as tf
import tensorflow.nn as nn

import sys
import time
# import warnings

# warnings.filterwarnings('once')

## Base de dados

In [2]:
ratings_titles = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings = pd.read_table(
    'ratings.dat',
    sep = '::',
    header = None,
    names = ratings_titles,
    engine = 'python'
)

ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
print(f'O dataset possui {ratings.shape[0]} instâncias.')

O dataset possui 1000209 instâncias.


In [4]:
num_users = ratings['user_id'].nunique()
num_movies = ratings['movie_id'].nunique()

print(f'Dessas instâncias, temos {num_users} usuários, e {num_movies} filmes.')

Dessas instâncias, temos 6040 usuários, e 3706 filmes.


## Preparando o modelo LSTM

### Hiperparâmetros

In [5]:
HYPERPARAM = {
    'batch_size': 50,
    'hidden_size': 128,
    'out_size': 64,
    'n_step': 1,
    'learning_rate': 0.01,
    'verbose': 10
}

### Placeholders

In [6]:
tf.disable_eager_execution()

user_id = tf.placeholder(tf.int32, shape = [None, 1], name = 'user_id')
movie_id = tf.placeholder(tf.int32, shape = [None, 1], name = 'movie_id')
rating = tf.placeholder(tf.float32, shape = [None, 1], name = 'rating')
dropout = tf.placeholder(tf.float32, name = 'dropout')

### Camadas

#### User embedding layer & lookup

In [7]:
with tf.variable_scope('user_id_embedding', reuse = tf.AUTO_REUSE):

    embedding_user = tf.get_variable(
        name = 'embedding_users',
        shape = [num_users, HYPERPARAM['hidden_size']],
        initializer = tf.glorot_uniform_initializer()
    )

    user_id_layer = nn.embedding_lookup(embedding_user, user_id)
    user_id_layer = nn.relu(user_id_layer)

#### Movie embedding layer & lookup

In [8]:
with tf.variable_scope('movie_id_embedding', reuse = tf.AUTO_REUSE):

    embedding_movie = tf.get_variable(
        name = 'embedding_movie',
        shape = [num_movies, HYPERPARAM['hidden_size']],
        initializer = tf.glorot_uniform_initializer()
    )

    movie_id_layer = nn.embedding_lookup(embedding_movie, movie_id)
    movie_id_layer = nn.relu(movie_id_layer)

#### User feedforward layer

#### Movie feedforward layer